# FastGen 入门实战 Notebook（代码内含详细中文注释）
作者：汪袁烁 

目标：用带注释的可运行示例 + 框架 demo，让你一边读注释一边跑代码，就能理解 FastGen / vLLM / DeepSpeed-MII 机制。


## 0. 环境准备与库导入

In [ ]:
# 安装所需库（在 notebook / 虚拟环境中执行）
# 如果你已经在环境中安装了部分，可以跳过或注释掉相应行

!pip install --upgrade pip

# PyTorch：根据你的 CUDA 版本选择对应的包，这里使用通用版本（可能是 CPU 或 GPU 版本）
!pip install torch torchvision torchaudio

# 用于 notebook 中画图
!pip install matplotlib

# DeepSpeed-MII，是 FastGen 的一部分，用于高性能推理  
!pip install deepspeed-mii

# vLLM，用于启用 chunked prefill 的示例  
!pip install vllm

# 如果你还打算尝试 transformers / huggingface 接口的话，也可以加上：
!pip install transformers

# （可选）如果用 tokenizers 或 accelerate 等辅助库，可额外安装
!pip install accelerate



首先，安装需要的库。

In [ ]:
import math, time, random
from dataclasses import dataclass
from typing import List, Optional, Dict
import torch
import torch.nn as nn
import torch.nn.functional as F

# 尝试导入 matplotlib 用于画图；若不可用则跳过画图功能
try:
    import matplotlib.pyplot as plt
    HAVE_MPL = True
except ImportError:
    print("matplotlib 不可用，图表将被跳过")
    HAVE_MPL = False

# 选择运行设备：如果有 GPU 则用 GPU，否则用 CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("使用设备：", device)

在这个 cell，我们导入 Python 和 PyTorch 的基础依赖，判断是否能画图，并确定计算设备（CPU 或 GPU）。

## 1. 构造极简解码器 + KV 缓存机制（带注释）
下面的代码定义一个非常简化的 Transformer 解码器层和语言模型，并显式管理 KV 缓存。

In [ ]:
class TinyDecoderLayer(nn.Module):
    def __init__(self, d_model=128, n_heads=4):
        super().__init__()
        self.d_model = d_model
        self.n_heads = n_heads
        self.d_head = d_model // n_heads  # 每个 head 的维度
        # Q, K, V 三个线性层
        self.q_proj = nn.Linear(d_model, d_model)
        self.k_proj = nn.Linear(d_model, d_model)
        self.v_proj = nn.Linear(d_model, d_model)
        # 输出映射层
        self.o_proj = nn.Linear(d_model, d_model)

    def forward(self, x, kv_cache: Optional[Dict[str, torch.Tensor]] = None):
        """
        参数：
          x: 输入张量形状 [B, T, D]
          kv_cache: 如果有历史缓存，则是 dict 包含 "k" 和 "v"，形状 [B, T_kv, H, Dh]
        返回：
          out: 输出张量 [B, T, D]
          new_cache: 最新的 KV 缓存
        """
        B, T, D = x.shape
        H, Dh = self.n_heads, self.d_head

        # 线性变换，得到 Q, K, V，并 reshape 为 [B, T, H, Dh]
        q = self.q_proj(x).view(B, T, H, Dh)
        k = self.k_proj(x).view(B, T, H, Dh)
        v = self.v_proj(x).view(B, T, H, Dh)

        # 如果有历史缓存，就把历史 K, V 拼接到当前 K, V 的前面
        if kv_cache is not None and "k" in kv_cache:
            k = torch.cat([kv_cache["k"], k], dim=1)
            v = torch.cat([kv_cache["v"], v], dim=1)

        # 构建新的缓存（detach 掉梯度），以供下步 decode 使用
        new_cache = {"k": k.detach(), "v": v.detach()}

        # 准备做 attention：reshape 为 [B*H, T, Dh]
        q_ = q.transpose(1, 2).contiguous().view(B*H, T, Dh)
        k_ = k.transpose(1, 2).contiguous().view(B*H, -1, Dh)
        v_ = v.transpose(1, 2).contiguous().view(B*H, -1, Dh)

        # 计算 attention 分数（q·k^T / sqrt(Dh)），然后 softmax
        attn = torch.bmm(q_, k_.transpose(1, 2)) / math.sqrt(Dh)
        attn = attn.softmax(dim=-1)
        # 用 attention 权重乘 v
        out = torch.bmm(attn, v_)

        # 还原形状到 [B, T, D]
        out = out.view(B, H, T, Dh).transpose(1, 2).contiguous().view(B, T, D)
        out = self.o_proj(out)

        return out, new_cache

class TinyLM(nn.Module):
    def __init__(self, vocab_size=2000, d_model=128, n_layers=2, n_heads=4):
        super().__init__()
        # embedding 层，将 token id 映射为向量
        self.emb = nn.Embedding(vocab_size, d_model)
        # 多个 decoder 层组成模型
        self.layers = nn.ModuleList([TinyDecoderLayer(d_model, n_heads)
                                     for _ in range(n_layers)])
        # 最后做线性映射得到 vocab 大小的 logits
        self.lm_head = nn.Linear(d_model, vocab_size, bias=False)

    @torch.no_grad()
    def forward_tokens(self, tokens: torch.Tensor,
                       kv_caches: Optional[List[Dict[str, torch.Tensor]]] = None):
        """
        参数：
          tokens: [B, T] 的 token id 序列
          kv_caches: 每层的历史缓存，形式为 list of dict
        返回：
          logits: [B, T, vocab_size]
          new_caches: 每层新的缓存列表
        """
        x = self.emb(tokens)
        new_caches = []
        for i, layer in enumerate(self.layers):
            kv = None if kv_caches is None else kv_caches[i]
            x, new_kv = layer(x, kv)
            new_caches.append(new_kv)
        logits = self.lm_head(x)
        return logits, new_caches

# 实例化模型，搬到指定设备，切换为 eval 模式（不启用 dropout 等）
model = TinyLM().to(device).eval()
print("vocab size:", model.lm_head.out_features)

这个代码块定义了解码器层 + 模型，并在模型中明确处理历史 KV 缓存拼接，注释清晰标出每一步的目的。

In [ ]:
@torch.no_grad()
def demo_prefill_decode(model, L_prompt=16, decode_steps=5, batch=1):
    # 初始时没有任何 KV 缓存
    kv_caches = None
    # 随机生成一个 prompt 序列，长为 L_prompt
    prompt = torch.randint(0, model.emb.num_embeddings,
                           (batch, L_prompt), device=device)
    # Prefill 阶段：一次性输入整个 prompt
    logits, kv_caches = model.forward_tokens(prompt, kv_caches=kv_caches)
    # 打印各层缓存中 k 的时间维度长度
    print("[Prefill] 各层 KV 长度：", [kv["k"].shape[1] for kv in kv_caches])

    # decode 阶段：一步一步地生成 token
    last = prompt[:, -1:].clone()
    for t in range(decode_steps):
        logits, kv_caches = model.forward_tokens(last, kv_caches=kv_caches)
        # 从 logits 中取最大得分 token 作为下一个输入
        next_tok = logits[:, -1, :].argmax(dim=-1, keepdim=True)
        last = next_tok
        print(f"[Decode 第 {t+1} 步] 各层 KV 长度：",
              [kv["k"].shape[1] for kv in kv_caches])

demo_prefill_decode(model, L_prompt=16, decode_steps=5, batch=1)

这个 cell 演示了 prefill 和 decode 的完整流程：
- Prefill：模型一次性“读入”整个 prompt，并生成初始 KV 缓存
- Decode：每一步输入上一步生成的 token，继续使用缓存并追加新的 KV
- 注释里解释每一步的操作意图，例如为何拼接缓存、为何取最大 token 等

## 2. 连续批处理（Continuous Batching）仿真（带注释）
在真实系统里，多条请求同时进入时，我们希望把 prefill 与 decode 混合调度以提升资源利用率。

启动 MII 服务

In [ ]:
import mii

# 选择一个中小模型便于测试
mii.serve(
    "facebook/opt-125m",
    profile_model_time=True,   # 开启各阶段耗时日志
    max_length=1024,
    tensor_parallel=1          # 如果你有多卡可设为 2、4…
)


客户端错峰请求演示

In [ ]:
import time
import threading
import mii

client = mii.client("facebook/opt-125m")

prompts = [
    "A: " + "hello " * 400 + "\nQ: Summarize in one sentence.",
    "Explain the significance of attention masks in Transformers.",
    "Write a haiku about GPUs."
]

def run_req(prompt, delay, name):
    time.sleep(delay)
    print(f"[{name}] send @+{delay:.1f}s")
    resp = client([prompt], max_new_tokens=64, stream=False)
    print(f"[{name}] done, text[:120]= {resp[0].generated_text[:120]!r}")

threads = [
    threading.Thread(target=run_req, args=(prompts[0], 0.0,   "R1-long")),
    threading.Thread(target=run_req, args=(prompts[1], 0.5,   "R2-mid")),
    threading.Thread(target=run_req, args=(prompts[2], 1.2,   "R3-short")),
]

for t in threads: t.start()
for t in threads: t.join()

print("All done.")


日志解析 + 绘图展示

In [ ]:
import re
import matplotlib.pyplot as plt

# 假设你把 MII 服务日志重定向到 “mii_server.log”
# 例如启动服务时用： python serve.py > mii_server.log 2>&1 &

logfile = "./fastgen_out.log"

# 正则要根据日志格式微调，下面是一个示例：
# 假设每 step 日志有这样一行：
#   “[Step=10] prefill_tokens=123 decode_tokens=456 …”
pattern = re.compile(r"\[Step=(\d+)\]\s+prefill_tokens=(\d+)\s+decode_tokens=(\d+)")

steps = []
prefill_tokens = []
decode_tokens = []

with open(logfile, 'r') as f:
    for line in f:
        m = pattern.search(line)
        if m:
            step = int(m.group(1))
            p = int(m.group(2))
            d = int(m.group(3))
            steps.append(step)
            prefill_tokens.append(p)
            decode_tokens.append(d)

plt.figure(figsize=(8, 4))
plt.plot(steps, prefill_tokens, label="prefill tokens / step")
plt.plot(steps, decode_tokens, label="decode tokens / step")
plt.xlabel("step index")
plt.ylabel("tokens")
plt.title("连续批处理中每 step 的 prefill / decode 分布")
plt.legend()
plt.tight_layout()
plt.show()


这一段代码仿真了“多请求同时到来”的场景：
- 定义 Request 对象保存每条请求的状态
- `step_continuous_batch` 按照三种策略分配预算
- `simulate_continuous_batch` 跑一轮并输出各策略结果
- 注释在代码中解释每一步为何这么做

## 3. chunked prefill（分块预填）仿真 + 注释
该部分展示切块预填如何在性能与资源占用之间做折中。

In [ ]:
@torch.no_grad()
def chunked_prefill_cost(model, L_prompt=2048, chunk_size=256, sleep_per_chunk_ms=5):
    # 计算需要多少块
    steps = math.ceil(L_prompt / chunk_size)
    # 随机生成一块输入作为真实前向
    dummy = torch.randint(0, model.emb.num_embeddings,
                          (1, min(chunk_size, L_prompt)), device=device)
    st = time.time()
    # 对第一块做真实前向 + 缓存计算
    logits, kv = model.forward_tokens(dummy, kv_caches=None)
    # 对剩余块，只做模拟开销（sleep），不真正计算
    for _ in range(steps - 1):
        if sleep_per_chunk_ms > 0:
            time.sleep(sleep_per_chunk_ms / 1000.0)
    et = time.time()
    return steps, et - st

for cs in [64, 128, 256, 512]:
    s, t = chunked_prefill_cost(model, L_prompt=2048,
                                 chunk_size=cs, sleep_per_chunk_ms=5)
    print(f"chunk_size = {cs:>4}, chunk 数 = {s:>3}, approx time = {t:.3f} 秒")

此 cell：
- 计算把 prompt 切块后需要多少块，以及模拟整体耗时
- 我们只对第一块做真实前向，其他块用 sleep 模拟开销，突出块数 / 大小对性能的影响
- 注释里标明为什么这样设计，以及每一步的目的

## 4. Dynamic SplitFuse 调度仿真 + 注释
这个部分是整个 notebook 的重头戏：仿真 Split + Fuse 调度策略。

In [ ]:
@dataclass
class Piece:
    rid: int
    is_last: bool
    size: int

def split_prompts(Ls: List[int], max_piece: int) -> List[Piece]:
    """
    把每个 prompt 拆成若干块 (piece)，每块大小 ≤ max_piece
    标记最后一块 is_last = True，以便调度允许 decode
    """
    pieces = []
    for rid, L in enumerate(Ls):
        full, rem = divmod(L, max_piece)
        for _ in range(full):
            pieces.append(Piece(rid, False, max_piece))
        if rem > 0:
            pieces.append(Piece(rid, True, rem))
        elif full > 0:
            # 如果刚好整除，则最后那块也标记为最后一块
            pieces[-1] = Piece(rid, True, max_piece)
        else:
            # prompt 长度为 0 的特殊情况，也标记为最后一块
            pieces.append(Piece(rid, True, 0))
    return pieces

def dynamic_split_fuse(L_prompts: List[int], L_generate: List[int],
                       budget: int, max_piece: int):
    """
    调度仿真：
    每 step 有固定 token 预算 budget。
    - Fuse：尽量把拆好的 prefill 块填满预算
    - Split：长 prompt 分块逐步 prefill
    - 只有属于最后一块的请求，允许用剩下预算做 decode
    返回三条历史记录：prefill_hist, decode_hist, piece_hist
    """
    from collections import deque
    pieces = split_prompts(L_prompts, max_piece)
    queue = deque(pieces)
    remain_gen = {i: g for i, g in enumerate(L_generate)}
    prefill_hist = []
    decode_hist = []
    piece_hist = []

    while len(queue) > 0 or any(v > 0 for v in remain_gen.values()):
        budget_left = budget
        used_pieces = []
        step_prefill = 0
        step_decode = 0
        last_piece_reqs = set()

        # Fuse 阶段：尽可能把多个 prefill 块塞满这个 step
        temp = deque()
        while budget_left > 0 and queue:
            p = queue.popleft()
            if p.size <= budget_left:
                used_pieces.append(p)
                budget_left -= p.size
                step_prefill += p.size
                if p.is_last:
                    last_piece_reqs.add(p.rid)
            else:
                temp.appendleft(p)
                break
        queue = temp + queue

        # 对于那些最后一块对应的请求，用剩余预算做 decode
        for rid in list(last_piece_reqs):
            if budget_left == 0:
                break
            need = remain_gen[rid]
            if need <= 0:
                continue
            use = min(need, budget_left)
            remain_gen[rid] -= use
            budget_left -= use
            step_decode += use

        prefill_hist.append(step_prefill)
        decode_hist.append(step_decode)
        piece_hist.append(len(used_pieces))
        # 如果本 step 完全没做任何事，就跳出避免死循环
        if step_prefill == 0 and step_decode == 0:
            break

    return prefill_hist, decode_hist, piece_hist

# 示例：用几条 prompt 长度差异大的请求来仿真
L_prompts = [1536, 128, 512, 4096, 64]
L_generate = [64, 64, 64, 64, 64]
budget = 1024
max_piece = 512

prefill_hist, decode_hist, piece_hist = dynamic_split_fuse(
    L_prompts, L_generate, budget, max_piece
)
print("总步数 =", len(prefill_hist),
      "总 prefill =", sum(prefill_hist),
      "总 decode =", sum(decode_hist))

if HAVE_MPL:
    xs = list(range(len(prefill_hist)))
    plt.figure(figsize=(8,4))
    plt.plot(xs, prefill_hist, label="prefill")
    plt.plot(xs, decode_hist, label="decode")
    plt.plot(xs, piece_hist, label="#pieces used")
    plt.xlabel("step")
    plt.ylabel("tokens / pieces")
    plt.title("Dynamic SplitFuse 调度仿真")
    plt.legend()
    plt.show()

这部分代码在做的事情：
- `split_prompts` 把 prompt 拆为块（piece），并标记最后一块
- `dynamic_split_fuse` 在每 step 内执行 Fuse + Split + decode 调度
- 保证拆、融合与 decode 共存，使得每一步 workload 较为平稳
- 注释中解释为何允许 decode 只在最后一块、为何先 Fuse 等细节

## 5. DeepSpeed-MII / vLLM 接口演示（带注释）
在你理解 toy 模型后，这里给出如何在实际库中调用 FastGen / chunked prefill 的示例。

In [ ]:
# DeepSpeed-MII 示例：快速上手 pipeline 接口（需安装 deepspeed-mii）
from mii import pipeline

# 以 Mistral-7B 模型为例构造 pipeline
pipe = pipeline("mistralai/Mistral-7B-v0.1")
# 给它两个 prompt，生成最多 64 个新 token
outputs = pipe(["你好，今天天气如何？", "请写一首诗"], max_new_tokens=64)
print("DeepSpeed-MII 输出：", outputs)

# vLLM 示例：启用 chunked prefill 功能（需安装 vllm）
from vllm import LLM
# 指定模型 + 开启 enable_chunked_prefill
llm = LLM(model="meta-llama/Llama-2-7b-hf", enable_chunked_prefill=True)
outs = llm.generate(["今天天气怎么样？", "写一首诗"], max_tokens=64)
print("vLLM 输出：", outs)

这个 cell 演示如何使用真实库：
- 用 DeepSpeed-MII 的 `pipeline` 接口启动推理服务
- 用 vLLM 启用 `enable_chunked_prefill` 特性进行生成
- 注释中解释每一步调用的意义，和我们前面 toy 模型对比